In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

In [2]:
df = sns.load_dataset('iris')
df.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [3]:
df['species'].unique()

array(['setosa', 'versicolor', 'virginica'], dtype=object)

In [4]:
df.isnull().sum()

sepal_length    0
sepal_width     0
petal_length    0
petal_width     0
species         0
dtype: int64

In [7]:
df = df[df['species']!="setosa"]

In [8]:
df.head()

,sepal_length,sepal_width,petal_length,petal_width,species
50,7.0,3.2,4.7,1.4,versicolor
51,6.4,3.2,4.5,1.5,versicolor
52,6.9,3.1,4.9,1.5,versicolor
53,5.5,2.3,4.0,1.3,versicolor
54,6.5,2.8,4.6,1.5,versicolor


In [11]:
df["species"] =  df['species'].map({
    "versicolor":0,
    "virginica":1
})

In [12]:
df.head()

,sepal_length,sepal_width,petal_length,petal_width,species
50,7.0,3.2,4.7,1.4,0
51,6.4,3.2,4.5,1.5,0
52,6.9,3.1,4.9,1.5,0
53,5.5,2.3,4.0,1.3,0
54,6.5,2.8,4.6,1.5,0


In [13]:
# Split dataset into dependent and independent dataset

X = df.drop(columns=['species'])
y = df['species']

In [14]:
# Train test split
from sklearn.model_selection import train_test_split

X_train, X_test, y_train , y_test = train_test_split(
    X, y, test_size=0.25, random_state=42
) 

In [15]:
from sklearn.linear_model import LogisticRegression

classifier = LogisticRegression()

In [16]:
from sklearn.model_selection import GridSearchCV
parameter = {
    'penalty':['l1', 'l2', 'elasticnet'], 'C':[1, 2, 3, 4, 5, 6, 7, 8 ,9 ,10, 20, 30, 40, 50, 55, 60], "max_iter":[10, 30, 50]
}

In [18]:
classifier_regressor = GridSearchCV(classifier, param_grid='parameter', scoring = 'accuracy', cv=5)

In [19]:
classifier_regressor.fit(X_train,y_train)

InvalidParameterError: The 'param_grid' parameter of GridSearchCV must be an instance of 'dict' or an instance of 'list'. Got 'parameter' instead.